In [5]:
import pandas as pd
import geopandas as gpd
import io
import json
import os

os.chdir("/Users/canyonfoot/Documents/python_proj/nwf-process-geodata")
metadata_df = pd.read_csv("output_metadata.csv")

In [6]:
metadata_df

,Unnamed: 0,New Category,NREX Category,Subtype,Person,Individual,Critical/Crucial Sublayer,Source,priority,notes,...,data_summary,source,format,geotype,file_api,homepage,data_link,metadata_link,use_constraints,s3_raw_path
0,0,Wildlife,Wildlife,Aquatic,Canyon,Crucial stream corridors,NaN,NREX,NaN,NaN,...,Data were developed to provide the location of...,data.geospatialhub.org/,geojson,vector,API,https://data.geospatialhub.org/datasets/b44ae3...,https://services6.arcgis.com/cWzdqIyxbijuhPLw/...,https://www.arcgis.com/sharing/rest/content/it...,The State of Wyoming and Wyoming Game and Fish...,data/raw/wildlife/aquatic/crucial_stream_corri...
1,1,Recreation,Wildlife,Aquatic,Canyon,Sport fish designations,NaN,NREX,NaN,NaN,...,This data set provides an indication of cold-w...,data.geospatialhub.org/,geojson,vector,API,https://data.geospatialhub.org/datasets/1bd26d...,https://services6.arcgis.com/cWzdqIyxbijuhPLw/...,https://www.arcgis.com/sharing/rest/content/it...,The State of Wyoming and Wyoming Game and Fish...,data/raw/recreation/aquatic/sport_fish_designa...
2,2,Wildlife,Wildlife,Big game herd units,Canyon,Antelope,NaN,NREX,High,NaN,...,This data set represents the 2022 antelope hun...,geospatialhub,geojson,vector,API,https://data.geospatialhub.org/datasets/fe86c4...,https://services6.arcgis.com/cWzdqIyxbijuhPLw/...,https://www.arcgis.com/sharing/rest/content/it...,The State of Wyoming and Wyoming Game and Fish...,data/raw/wildlife/big_game_herd_units/antelope...
3,3,Wildlife,Wildlife,Big game herd units,Canyon,Bighorn sheep,NaN,NREX,High,NaN,...,This data set represents the 2023 bighorn shee...,geospatialhub,geojson,vector,API,https://data.geospatialhub.org/datasets/e88689...,https://services6.arcgis.com/cWzdqIyxbijuhPLw/...,https://www.arcgis.com/sharing/rest/content/it...,The State of Wyoming and Wyoming Game and Fish...,data/raw/wildlife/big_game_herd_units/bighorn_...
4,4,Wildlife,Wildlife,Big game herd units,Canyon,Bison,NaN,NREX,High,NaN,...,This data set represents the 2016 bison huntar...,geospatialhub,geojson,vector,API,https://data.geospatialhub.org/datasets/50a7f0...,https://services6.arcgis.com/cWzdqIyxbijuhPLw/...,https://www.arcgis.com/sharing/rest/content/it...,The State of Wyoming and Wyoming Game and Fish...,data/raw/wildlife/big_game_herd_units/bison.ge...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,113,Climate Change Indicators,Climate Features,Drought,Canyon,LOCA Extreme Temperature Forecast,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
114,114,Climate Change Indicators,Climate Features,Flooding,Canyon,FEMA NFHL,NaN,NaN,NaN,NaN,...,National Flood Hazard Layer (NFHL) dataset is ...,hazards.fema.gov,gdb,NaN,NaN,https://hazards.fema.gov/femaportal/NFHL/searc...,https://hazards.fema.gov/femaportal/NFHL/Downl...,NaN,NaN,data/raw/climate_change_indicators/flooding/fe...
115,115,Climate Change Indicators,Climate Features,Extreme heat,Canyon,NaN,NaN,NaN,NaN,NaN,...,The Localized Constructed Analogs (LOCA) data ...,https://atlas.globalchange.gov/,tar.gz,raster,file,https://atlas.globalchange.gov/pages/nca4archive,https://downloads.globalchange.gov/scenarios/t...,NaN,NaN,NaN
116,116,Administrative,Congressional Districts,Representatives,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
from src.download import DataDownloaderUploader
from src.process import Processor
from src.document import DataDocumenter
from src.export import s3Exporter

In [8]:
DataDownloaderUploader().download_and_upload_main()

118it [00:04, 23.61it/s]

Successfully uploaded data/raw/climate_change_indicators/extreme_heat/loca_extreme_temperature_forecast.tar.gz


In [ ]:
DataDownloaderUploader().download_and_upload_main()
Processor().process_raw_datasets()
DataDocumenter().document_processed_data()
s3Exporter().export_all("export_test")

In [ ]:
s3Exporter().export_all("export_test")

In [ ]:
from fpdf import FPDF

# Create instance of FPDF class
pdf = FPDF()

# Add a page
pdf.add_page()

# Set font
pdf.set_font("Arial", size=12)

# Add a cell
pdf.cell(200, 10, txt="Welcome to PyFPDF!", ln=True, align='R')

# Save the PDF to a file
pdf.output("simple_demo.pdf")

In [ ]:
def read_gdb_file():
    extracted_folder_path = f"{city_path}/floodmaps/{FEMA_file_prefix}"
    gdb_folder_path = f"{extracted_folder_path}/{FEMA_file_prefix}.gdb"

    if not os.path.exists(extracted_folder_path):
        extract_zip_file(zip_path, extracted_folder_path)

    gdb_table_to_read = find_largest_file(gdb_folder_path)
    FEMA_flood = geopd.read_file(gdb_table_to_read)

    return FEMA_flood


def process_floodmap(config, msa_id):
    base_path = config["base_path"]
    FEMA_file_prefix = config[msa_id]["FEMA_file_name"]
    city_path = f"{base_path}/cities/{msa_id}"
    zip_path = f"{base_path}/national/floodmaps/{FEMA_file_prefix}.zip"
    extracted_folder_path = f"{city_path}/floodmaps/{FEMA_file_prefix}"
    gdb_folder_path = f"{extracted_folder_path}/{FEMA_file_prefix}.gdb"
    tract_path = f"{city_path}/census/geo/tracts.geojson"

    if not os.path.exists(extracted_folder_path):
        extract_zip_file(zip_path, extracted_folder_path)

    gdb_table_to_read = find_largest_file(gdb_folder_path)

    FEMA_flood = geopd.read_file(gdb_table_to_read)
    FEMA_flood = FEMA_flood.to_crs(4326)
    FEMA_flood["geometry"] = FEMA_flood.simplify(0.0001).buffer(0)

    tracts_combined = geopd.GeoDataFrame({'geometry' : [geopd.read_file(tract_path).unary_union]}, index=[0]).set_crs(4326)

    out = FEMA_flood.overlay(tracts_combined)
    out.to_file(f"{city_path}/floodmaps/processed_fema.geojson")
    print(f"Processed FEMA floodmap data written to: {city_path}/floodmaps/processed_fema.geojson") 

In [ ]:
gpd.read_file("https://hazards.fema.gov/femaportal/NFHL/Download/ProductsDownLoadServlet?DFIRMID=56&state=WYOMING&county=&fileName=NFHL_56_20221104.zip")